In [ ]:
%matplotlib inline
#In this script, I'm going to generate a random walk model.  This is my first random walk model,
#and is really simple.  Basically, it starts with evidence of 0 and randomly drifts
# with a step size that is a normal gaussian with .3 std dev and a criterion of 3.

#After running a number of random walks, I can plot a subsection of those to visualize results.
#To do that, I will use matplotlib and seaborn (which sits on top of matplotlib)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##Random Walk Model
nreps=100 #How many repititions (or trials or whatever)
nsamples=2000 #How many "Steps" are possible

drift=0.0 #Non-informative stimulus (no positive or negative mean step influence)
sdrw = 0.3 #This is the sd for the random walk
criterion = 3.0 #Evidence accumulation for decision

#Create blank vectors/matrices for storing model info
latencies=np.repeat(0,nreps)
responses=np.repeat(0,nreps)
evidence=np.zeros((nreps,nsamples+1))

#This is where we actually run the random walk model
for i in range(nreps):
    #We just create a cumulative sum of random normals generated for the nsamples possible.
    evidence[i,:]=np.cumsum(np.append(0,np.random.normal(drift,sdrw,nsamples)))
    #then See when (if) that accumulation crosses our criterion
    p=np.argmax(abs(evidence[i,:])>criterion)
    #Record whether that was a positive or negative threshold cross (so decision 1 or 2)
    responses[i]=np.sign(evidence[i,p])
    #Record the latency (how many "steps" to decision)
    latencies[i]=p
    

In [ ]:
#Plot a few example walks.
#plt.hold(True)
plt.title('A Few random walks')
plt.ylabel('Evidence')
plt.xlabel('time')
#draw arbitrary lines on the plot
xLen=np.max(latencies[0:3])
plt.plot(range(xLen),np.repeat(-3,len(range(xLen))),'k--')
plt.plot(range(xLen),np.repeat(3,len(range(xLen))),'k--')
#Now plot the first four random walks.  Though there are many more
plt.plot(range(latencies[0]),evidence[0,0:latencies[0]],'b--')
plt.plot(range(latencies[1]),evidence[1,0:latencies[1]],'r-.')
plt.plot(range(latencies[2]),evidence[2,0:latencies[2]],'g')
plt.plot(range(latencies[3]),evidence[3,0:latencies[3]],'y:')

In [ ]:
#We can also plot a histogram of all of the latencies from these random walks. 
#That allows us to see the distribution of decision "RTs"
#Seaborn distplot makes this super easy.
sns.distplot(latencies)

#### Now let's add a little bit of a wrinkle just to see how it impacts the model

In [ ]:
#This version extends the original model by allowing for a couple of changes.
#   1) We are adding a slight trial by trial bias to the starting point (t2tsd[0])
#   2) We are adding a positive bias to the drift rate (drift=0.03)
#   3) we are allowing for drift rate to vary from trial to trial (t2tsd[1])

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##Random Walk Model
nreps=1000
nsamples=2000

drift=0.03 #Drift rate is slightly positive, so we are biasing towards decision 1
sdrw = 0.3 #set the standard dev of the drift rate
criterion = 3.0 #Set the critierion

#New introduction of trial to trial variability
t2tsd=np.array([0.0,0.025]) 

latencies=np.repeat(0,nreps)
responses=np.repeat(0,nreps)
evidence=np.zeros((nreps,nsamples+1))

for i in range(nreps):
    #Set the starting point as varying around 0 with some sd set above.
    sp=np.random.normal(0,t2tsd[0],1)
    #Set the drift rate mean by selecting something varying the bias above and our bias std set above
    dr=np.random.normal(drift,t2tsd[1],1)
    evidence[i,:]=np.cumsum(np.append(sp,np.random.normal(dr,sdrw,nsamples)))
    p=np.argmax(abs(evidence[i,:])>criterion)
    responses[i]=np.sign(evidence[i,p])
    latencies[i]=p


In [ ]:
#Plot a few example walks.
plt.hold(True)
plt.title('A Few random walks')
plt.ylabel('Evidence')
plt.xlabel('time')
#draw arbitrary lines on the plot
xLen=np.max(latencies[0:3])
plt.plot(range(xLen),np.repeat(-3,len(range(xLen))),'k--')
plt.plot(range(xLen),np.repeat(3,len(range(xLen))),'k--')
#Draw the walks on the graph
plt.plot(range(latencies[0]),evidence[0,0:latencies[0]],'b--')
plt.plot(range(latencies[1]),evidence[1,0:latencies[1]],'r-.')
plt.plot(range(latencies[2]),evidence[2,0:latencies[2]],'g')
plt.plot(range(latencies[3]),evidence[3,0:latencies[3]],'y:')

In [ ]:
#Look at histogram of responses for each type of response
topResp=latencies[responses>0]
botResp=latencies[responses<0]

print('The number of top Responses is '+np.str(np.shape(topResp)[0]) )
print('The mean response time for top Resps is '+np.str(np.mean(topResp)))
#sns.distplot(topResp)

In [ ]:
print('The number of bottom Responses is '+np.str(np.shape(botResp)[0]) )
print('The mean response time for bottom Resps is '+np.str(np.mean(botResp)))

So great, we can see that responses are clearly biased towards the top response, for instance the correct response, and away from the bottom response.

In [ ]:
def driftFunc(drift,sdrw,criterion,nReps,nSamples,driftVar):
    #Ok this is the functionized version, where I have to input the params of interest.
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    ##Random Walk Model
    nreps=nReps
    nsamples=nSamples

    #drift=drift#0.03 #Drift rate is slightly positive, so we are biasing towards decision 1
    #sdrw = 0.3 #set the standard dev of the drift rate
    #criterion = 3.0 #Set the critierion

    #New introduction of trial to trial variability
    t2tsd=driftVar

    latencies=np.repeat(0,nreps)
    responses=np.repeat(0,nreps)
    evidence=np.zeros((nreps,nsamples+1))

    for i in range(nreps):
        #Set the starting point as varying around 0 with some sd set above.
        sp=np.random.normal(0,t2tsd[0],1)
        #Set the drift rate mean by selecting something varying the bias above and our bias std set above
        dr=np.random.normal(drift,t2tsd[1],1)
        evidence[i,:]=np.cumsum(np.append(sp,np.random.normal(dr,sdrw,nsamples)))
        p=np.argmax(abs(evidence[i,:])>criterion)
        responses[i]=np.sign(evidence[i,p])
        latencies[i]=p
        
    return responses, latencies, evidence
    #return latencies

In [ ]:
#Now we can play around with values here.  In this example, I wanted to have a version
#where the top response was biased towards and the starting variability was large. This sort
#mimics a situation in a go/no-go experiment where there could be a strong bias towards one response
#occuring or being "correct" more often.
responses2, latencies2, evidence2= driftFunc(0.035,.3,3.0,1000,2000,[0.8,0])

In [ ]:
#Plot a few example walks.
plt.title('A Few random walks')
plt.ylabel('Evidence')
plt.xlabel('time')
#draw arbitrary lines on the plot
xLen=np.max(latencies2[0:3])
plt.plot(range(xLen),np.repeat(-3,len(range(xLen))),'k--')
plt.plot(range(xLen),np.repeat(3,len(range(xLen))),'k--')
#Draw the walks on the graph
plt.plot(range(latencies2[0]),evidence2[0,0:latencies2[0]],'b--')
plt.plot(range(latencies2[1]),evidence2[1,0:latencies2[1]],'r-.')
plt.plot(range(latencies2[2]),evidence2[2,0:latencies2[2]],'g')
plt.plot(range(latencies2[3]),evidence2[3,0:latencies2[3]],'y:')

In [ ]:
#Look at histogram of responses for each type of response
topResp=latencies2[responses2>0]
botResp=latencies2[responses2<0]

print('The number of top Responses is '+np.str(np.shape(topResp)[0]) )
print('The mean response time for top Resps is '+np.str(np.mean(topResp)))


In [ ]:
print('The number of bottom Responses is '+np.str(np.shape(botResp)[0]) )
print('The mean response time for bottom Resps is '+np.str(np.mean(botResp)))

You'll notice much fewer response that went to the bottom response, and when they did, they were faster than the correct responses.

In [ ]:
#Another example
responses2, latencies2, evidence2= driftFunc(0.035,.3,3.0,1000,2000,[0.0,0.025])

In [ ]:
topResp=latencies2[responses2>0]
botResp=latencies2[responses2<0]

print('The number of top Responses is '+np.str(np.shape(topResp)[0]) )
print('The mean response time for top Resps is '+np.str(np.mean(topResp)))
print('The number of bottom Responses is '+np.str(np.shape(botResp)[0]) )
print('The mean response time for bottom Resps is '+np.str(np.mean(botResp)))

In this version, there is still a clear impact of starting point bias. However, I've allowed for the drift rate to vary on each iteration. So now we see slower RTs for "incorrect" bottom decisions as compared to the top decision.  That is because on average the bias + drift rate means that there can be more negative boundary crossings but they are on average going to be slower than correct response.

In [ ]:
#We can flip that relationship by flipping the drift rate mean.
responses2, latencies2, evidence2= driftFunc(-0.035,.3,3.0,1000,2000,[0.0,0.025])

In [ ]:
topResp=latencies2[responses2>0]
botResp=latencies2[responses2<0]

print('The number of top Responses is '+np.str(np.shape(topResp)[0]) )
print('The mean response time for top Resps is '+np.str(np.mean(topResp)))
print('The number of bottom Responses is '+np.str(np.shape(botResp)[0]) )
print('The mean response time for bottom Resps is '+np.str(np.mean(botResp)))

In [ ]:
#We can flip that relationship by flipping the drift rate mean.
responses2, latencies2, evidence2= driftFunc(0.0,.3,3.0,1000,2000,[0.0,0.0])

In [ ]:
topResp=latencies2[responses2>0]
botResp=latencies2[responses2<0]

print('The number of top Responses is '+np.str(np.shape(topResp)[0]) )
print('The mean response time for top Resps is '+np.str(np.mean(topResp)))
print('The number of bottom Responses is '+np.str(np.shape(botResp)[0]) )
print('The mean response time for bottom Resps is '+np.str(np.mean(botResp)))

In [ ]:
#Again all else being equal-